<a href="https://colab.research.google.com/github/Phoenix-7Theta-Fi/AridWigglyTransformations/blob/main/2nd_October.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Install required libraries
!pip install langchain
!pip install langchain-community
!pip install -q langchain-google-genai langchain-qdrant qdrant-client
!pip install protobuf==3.20.3

In [2]:
# Required imports
from langchain_google_genai import GoogleGenerativeAI
import os
import getpass

# Set the Google API key
if "GOOGLE_API_KEY" not in os.environ:
    os.environ["GOOGLE_API_KEY"] = "qwertyuiop"

# Initialize Google Generative AI (Gemini 1.5 Pro) for both Triage and Diagnostic agents
triage_agent = GoogleGenerativeAI(model="gemini-1.5-pro", google_api_key=os.environ["GOOGLE_API_KEY"])
diagnostic_agent = GoogleGenerativeAI(model="gemini-1.5-pro", google_api_key=os.environ["GOOGLE_API_KEY"])


In [3]:
# Required imports for Qdrant and Google embeddings
from qdrant_client import QdrantClient
from langchain_qdrant import QdrantVectorStore
from langchain_google_genai import GoogleGenerativeAIEmbeddings
import os

# Set the Google API key and Qdrant API key directly from the provided credentials
os.environ["GOOGLE_API_KEY"] = "qwertyuiop"  # Provided Google API key

QDRANT_API_KEY = "abcdefghi"
QDRANT_URL = "abcdefg.com"  # Provided Qdrant URL

# Initialize Qdrant client using the cloud URL and API key
client = QdrantClient(url=QDRANT_URL, api_key=QDRANT_API_KEY)

# Initialize Google Generative AI Embeddings model for vectorization
embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")

# Use the existing "ayurveda_collection" in Qdrant
vector_store = QdrantVectorStore(
    client=client,
    collection_name="ayurveda_collection",  # Provided collection name
    embedding=embeddings,  # Embedding model for semantic search
)


In [4]:
from langchain.memory import ConversationBufferMemory

# Initialize conversation buffer for managing consultation context
conversation_memory = ConversationBufferMemory()


<ipython-input-4-62dd46c0a1dd>:4: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  conversation_memory = ConversationBufferMemory()
